In [54]:
# SPMF - try to get most occurring sequences
from spmf import Spmf
import pandas as pd
from text_cleaner import *
from tqdm import tqdm
import itertools

## Loading the dataset and cleaning the text

In [3]:
# Load the Twitter dataset
twitter_df = pd.read_csv('tweets_28_02_2021.csv', index_col=0)

# Print the head of the loaded dataset
twitter_df.head()

_id  \
0  5f9f1c36b38e10f823bf2cef   
1  5f9f1c36b38e10f823bf2ce7   
2  5f9f1c36b38e10f823bf2d0a   
3  5f9f1c36b38e10f823bf2cf5   
4  5f9f1c36b38e10f823bf2d07   

                                          tweet_text    username  \
0               @eliostruyf So exciting, have fun! 😊  LEGO_Group   
1  These Brick-O-Lanterns are certainly all treat...  LEGO_Group   
2  @dentistescabri Nous prenons la sécurité de no...  LEGO_Group   
3                                @Jasmin80212446 😍🎄🥰  LEGO_Group   
4  @ashleydrixey Sounds like a perfect fit for th...  LEGO_Group   

                created_at timestamp archetype  
0  2020-10-30 18:23:50.000       NaN    artist  
1  2020-10-31 09:00:28.000       NaN    artist  
2  2020-10-30 12:07:58.000       NaN    artist  
3  2020-10-30 16:35:39.000       NaN    artist  
4  2020-10-30 13:09:14.000       NaN    artist

In [4]:
# Clean-up the texts
twitter_df["cleaned_text"] = twitter_df["tweet_text"].apply(lambda x: clean_up_text(x))

# Tokenize the cleaned texts
twitter_df["cleaned_text"] = twitter_df["cleaned_text"].apply(lambda x: nltk.word_tokenize(x))

# Remove the stopwords
twitter_df["cleaned_text"] = twitter_df["cleaned_text"].apply(lambda x: remove_stopwords(x))

# Drop the rows with empty 'cleaned_text' field
twitter_df = twitter_df.drop(twitter_df[twitter_df['cleaned_text'].map(len) < 1].index)

# Print the new head of the dataset
twitter_df.head()

_id  \
0  5f9f1c36b38e10f823bf2cef   
1  5f9f1c36b38e10f823bf2ce7   
2  5f9f1c36b38e10f823bf2d0a   
4  5f9f1c36b38e10f823bf2d07   
5  5f9f1c36b38e10f823bf2d08   

                                          tweet_text    username  \
0               @eliostruyf So exciting, have fun! 😊  LEGO_Group   
1  These Brick-O-Lanterns are certainly all treat...  LEGO_Group   
2  @dentistescabri Nous prenons la sécurité de no...  LEGO_Group   
4  @ashleydrixey Sounds like a perfect fit for th...  LEGO_Group   
5  @irgator04 What a perfect way to start your we...  LEGO_Group   

                created_at timestamp archetype  \
0  2020-10-30 18:23:50.000       NaN    artist   
1  2020-10-31 09:00:28.000       NaN    artist   
2  2020-10-30 12:07:58.000       NaN    artist   
4  2020-10-30 13:09:14.000       NaN    artist   
5  2020-10-30 12:49:37.000       NaN    artist   

                                        cleaned_text  
0                                    [exciting, fun]  
1  [brick, lanterns, certainly, treat, trick, get...  
2  [nous, prenons, la, curit, de, nos, fans, tr, ...  
4  [sounds, like, perfect, fit, aspiring, young, ...  
5                     [perfect, way, start, weekend]

## Testing SPMF on 'artist' archetype

In [5]:
# Example 1 - try to get the most occurring words in the 'artist' archetype subset
# Extract all the tweets for the 'artist' archetype
artist_df = twitter_df.cleaned_text[twitter_df["archetype"] == "artist"]

# Reset the index of the subset
artist_df = artist_df.reset_index(drop=True)

# Print the head of the subset
artist_df.head()

0                                      [exciting, fun]
1    [brick, lanterns, certainly, treat, trick, get...
2    [nous, prenons, la, curit, de, nos, fans, tr, ...
3    [sounds, like, perfect, fit, aspiring, young, ...
4                       [perfect, way, start, weekend]
Name: cleaned_text, dtype: object

In [31]:
# Create a list of sentences
artist_list = [" ".join(row) for row in artist_df.tolist()]

In [36]:
# SPMF - get the most frequent sequences
spmf = Spmf("PrefixSpan", input_direct=artist_list,
            output_filename="output.txt", arguments=[0.001, 3], input_type="text")
spmf.run()
print(spmf.to_pandas_dataframe(pickle=True))
spmf.to_csv("output.csv")

>/mnt/HDD_Linux/Praca_magisterska/jupyter_notebooks/data_mining/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 328 ms
 Frequent sequences count : 13662
 Max memory (mb) : 126.75393676757812
 minsup = 12 sequences.
 Pattern count : 13662

Post-processing to show result in terms of string values.
Post-processing completed.

                   pattern  sup
0               [exciting]   46
1                    [fun]  171
2               [fun, get]   14
3          [fun, building]   20
4         [fun, available]   12
...                    ...  ...
13657       [lunarnewyear]   13
13658  [blackhistorymonth]   12
13659       [goldenglobes]   14
13660     [nintendodirect]   62
13661             [sinnoh]   12

[13662 rows x 2 columns]


# Calculating most frequent sequences for all of the archetypes

In [65]:
# Generating 
archetype_list = ['artist',
                 'caregiver',
                 'everyman',
                 'explorer',
                 'guru',
                 'hero',
                 'innocent',
                 'jester',
                 'magician',
                 'rebel',
                 'ruler',
                 'seducer']

full_df = pd.DataFrame(columns=['chunk'] + archetype_list)

# Iterate over archetypes
for archetype in tqdm(archetype_list):
    # Extract all the tweets for the 'artist' archetype
    tmp_df = twitter_df.cleaned_text[twitter_df["archetype"] == archetype]

    # Reset the index of the subset
    tmp_df = tmp_df.reset_index(drop=True)
    
    # Calculate number of words, number of two-word and three-word combinations
    unique_words = set()
    unique_two_words = set()
    unique_three_words = set()
    
    for row in tmp_df.tolist():
        unique_words.update(row)
        unique_two_words.update(itertools.permutations(row, 2))
        unique_three_words.update(itertools.permutations(row, 3))
    
    cnt_single_word = len(unique_words)
    cnt_two_words = len(unique_two_words)
    cnt_three_words = len(unique_three_words)
    
    # Create a list of sentences
    tmp_list = [" ".join(row) for row in tmp_df.tolist()]
    
    # SPMF - get the most frequent sequences
    spmf = Spmf("PrefixSpan", input_direct=tmp_list,
                output_filename=f"sequence_files/output_{archetype}.txt", arguments=[0.001, 3], input_type="text")
    spmf.run()
    
    spmf = spmf.to_pandas_dataframe(pickle=True)
    
    # Get the TF
    spmf.sup[spmf.pattern.map(len) == 1] = spmf.sup[spmf.pattern.map(len) == 1].apply(lambda x: float(x / cnt_single_word))
    spmf.sup[spmf.pattern.map(len) == 2] = spmf.sup[spmf.pattern.map(len) == 2].apply(lambda x: float(x / cnt_two_words)) 
    spmf.sup[spmf.pattern.map(len) == 3] = spmf.sup[spmf.pattern.map(len) == 3].apply(lambda x: float(x / cnt_three_words)) 
    
    print(spmf)
    spmf.to_csv(f"sequence_files/output_{archetype}.csv")

  0%|          | 0/12 [00:00<?, ?it/s]<ipython-input-65-1751e09f43bc>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spmf.sup[spmf.pattern.map(len) == 1] = spmf.sup[spmf.pattern.map(len) == 1].apply(lambda x: float(x / cnt_single_word))
  8%|▊         | 1/12 [00:13<02:33, 13.95s/it]

>/mnt/HDD_Linux/Praca_magisterska/jupyter_notebooks/data_mining/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 288 ms
 Frequent sequences count : 13662
 Max memory (mb) : 126.65481567382812
 minsup = 12 sequences.
 Pattern count : 13662

Post-processing to show result in terms of string values.
Post-processing completed.

                   pattern       sup
0               [exciting]  0.004242
1                    [fun]  0.015768
2               [fun, get]  0.000018
3          [fun, building]  0.000026
4         [fun, available]  0.000015
...                    ...       ...
13657       [lunarnewyear]  0.001199
13658  [blackhistorymonth]  0.001107
13659       [goldenglobes]  0.001291
13660     [nintendodirect]  0.005717
13661             [sinnoh]  0.001107

[13662 rows x 2 columns]
>/mnt/HDD_Linux/Praca_magisterska/jupyter_notebooks/data_mining/spmf.jar
Converting TEXT to SPMF format.
Conversi

<ipython-input-65-1751e09f43bc>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spmf.sup[spmf.pattern.map(len) == 1] = spmf.sup[spmf.pattern.map(len) == 1].apply(lambda x: float(x / cnt_single_word))
 17%|█▋        | 2/12 [00:26<02:11, 13.20s/it]

                    pattern           sup
0                   [great]  1.425055e-02
1             [great, news]  1.833057e-05
2               [great, us]  3.462442e-05
3           [great, us, us]  6.287541e-07
4         [great, us, form]  6.287541e-07
...                     ...           ...
47059               [korea]  1.036404e-03
47060            [eritrean]  1.813706e-03
47061  [eritrean, soldiers]  1.425711e-05
47062                [view]  1.036404e-03
47063                [axum]  1.165954e-03

[47064 rows x 2 columns]
>/mnt/HDD_Linux/Praca_magisterska/jupyter_notebooks/data_mining/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 222 ms
 Frequent sequences count : 23948
 Max memory (mb) : 100.57962799072266
 minsup = 9 sequences.
 Pattern count : 23948

Post-processing to show result in terms of string values.
Post-processing completed.



<ipython-input-65-1751e09f43bc>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spmf.sup[spmf.pattern.map(len) == 1] = spmf.sup[spmf.pattern.map(len) == 1].apply(lambda x: float(x / cnt_single_word))
 25%|██▌       | 3/12 [00:34<01:37, 10.80s/it]

...

<ipython-input-65-1751e09f43bc>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spmf.sup[spmf.pattern.map(len) == 1] = spmf.sup[spmf.pattern.map(len) == 1].apply(lambda x: float(x / cnt_single_word))
 92%|█████████▏| 11/12 [02:17<00:11, 11.82s/it]

                          pattern           sup
0                           [aww]  9.980040e-04
1                         [thank]  3.629105e-02
2                   [thank, much]  3.595630e-05
3               [thank, much, us]  7.855333e-07
4                   [thank, kind]  1.926231e-05
...                           ...           ...
16644  [blackhistorymonth, black]  1.284154e-05
16645               [mckinseybhm]  1.542370e-03
16646                       [bhm]  9.072764e-04
16647                 [valentine]  1.179459e-03
16648             [valentinesday]  1.088732e-03

[16649 rows x 2 columns]
>/mnt/HDD_Linux/Praca_magisterska/jupyter_notebooks/data_mining/spmf.jar
Converting TEXT to SPMF format.
Conversion completed.
=============  PREFIXSPAN 0.99-2016 - STATISTICS =============
 Total time ~ 255 ms
 Frequent sequences count : 71403
 Max memory (mb) : 104.82957458496094
 minsup = 4 sequences.
 Pattern count : 71403

Post-processing to show result in terms of string values.
Post-proce

<ipython-input-65-1751e09f43bc>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spmf.sup[spmf.pattern.map(len) == 1] = spmf.sup[spmf.pattern.map(len) == 1].apply(lambda x: float(x / cnt_single_word))


                             pattern           sup
0                            [story]  7.396915e-03
1                       [story, kim]  1.126215e-05
2                [story, kim, jones]  4.750281e-07
3                     [story, jones]  1.126215e-05
4                        [story, us]  1.970876e-05
...                              ...           ...
71398  [beauvais, arthur, kersauson]  4.750281e-07
71399         [beauvais, arthur, de]  4.750281e-07
71400          [beauvais, kersauson]  1.126215e-05
71401                 [beauvais, de]  1.126215e-05
71402      [beauvais, de, kersauson]  4.750281e-07

[71403 rows x 2 columns]


100%|██████████| 12/12 [02:27<00:00, 12.33s/it]


In [74]:
# Merge all files into a single dataframe
total_df = pd.DataFrame(columns=["pattern"] + archetype_list)
for archetype in tqdm(archetype_list):
    # Read file
    new_df = pd.read_csv(f"sequence_files/output_{archetype}.csv", sep=",", index_col=0)
    new_df = new_df.rename(columns={"sup": archetype})
    
    # Merge it with the other ones
    frames = [total_df, new_df]
    total_df = pd.concat(frames, ignore_index=True)

100%|██████████| 12/12 [00:00<00:00, 12.91it/s]


In [75]:
# Print dataframe
total_df

pattern    artist caregiver everyman  \
0                              ['exciting']  0.004242       NaN      NaN   
1                                   ['fun']  0.015768       NaN      NaN   
2                            ['fun', 'get']  0.000018       NaN      NaN   
3                       ['fun', 'building']  0.000026       NaN      NaN   
4                      ['fun', 'available']  0.000015       NaN      NaN   
...                                     ...       ...       ...      ...   
361448  ['beauvais', 'arthur', 'kersauson']       NaN       NaN      NaN   
361449         ['beauvais', 'arthur', 'de']       NaN       NaN      NaN   
361450            ['beauvais', 'kersauson']       NaN       NaN      NaN   
361451                   ['beauvais', 'de']       NaN       NaN      NaN   
361452      ['beauvais', 'de', 'kersauson']       NaN       NaN      NaN   

       explorer guru hero innocent jester magician rebel ruler      seducer  
0           NaN  NaN  NaN      NaN    NaN      NaN   NaN   NaN          NaN  
1           NaN  NaN  NaN      NaN    NaN      NaN   NaN   NaN          NaN  
2           NaN  NaN  NaN      NaN    NaN      NaN   NaN   NaN          NaN  
3           NaN  NaN  NaN      NaN    NaN      NaN   NaN   NaN          NaN  
4           NaN  NaN  NaN      NaN    NaN      NaN   NaN   NaN          NaN  
...         ...  ...  ...      ...    ...      ...   ...   ...          ...  
361448      NaN  NaN  NaN      NaN    NaN      NaN   NaN   NaN  4.75028e-07  
361449      NaN  NaN  NaN      NaN    NaN      NaN   NaN   NaN  4.75028e-07  
361450      NaN  NaN  NaN      NaN    NaN      NaN   NaN   NaN  1.12621e-05  
361451      NaN  NaN  NaN      NaN    NaN      NaN   NaN   NaN  1.12621e-05  
361452      NaN  NaN  NaN      NaN    NaN      NaN   NaN   NaN  4.75028e-07  

[361453 rows x 13 columns]

In [77]:
aggregate_func = {
    "pattern": "first",
    "artist": "sum",
    "caregiver": "sum",
    "everyman": "sum",
    "explorer": "sum",
    "guru": "sum",
    "hero": "sum",
    "innocent": "sum",
    "jester": "sum",
    "magician": "sum",
    "rebel": "sum",
    "ruler": "sum",
    "seducer": "sum"
}
total_df = total_df.groupby("pattern").aggregate(aggregate_func)
total_df = total_df.reset_index(drop=True)

In [78]:
# Print the full DataFrame
total_df

pattern    artist  caregiver  everyman  explorer      guru  \
0                ['aaron']  0.000000   0.000000       0.0  0.000786  0.000000   
1                ['abbie']  0.000000   0.000000       0.0  0.000000  0.000000   
2              ['ability']  0.000000   0.000000       0.0  0.001072  0.000793   
3         ['able', 'able']  0.000042   0.000000       0.0  0.000000  0.000000   
4       ['able', 'access']  0.000000   0.000000       0.0  0.000000  0.000000   
...                    ...       ...        ...       ...       ...       ...   
305816             ['zip']  0.000000   0.003239       0.0  0.000000  0.000000   
305817            ['zone']  0.001107   0.000000       0.0  0.000000  0.000000   
305818            ['zoom']  0.002029   0.000000       0.0  0.000000  0.001220   
305819              ['zu']  0.000000   0.000000       0.0  0.000000  0.000000   
305820             ['zum']  0.000000   0.000000       0.0  0.000000  0.000000   

            hero  innocent    jester  magician     rebel     ruler  seducer  
0       0.000653  0.000000  0.000000  0.000000  0.000000  0.000000      0.0  
1       0.000000  0.000000  0.000000  0.003622  0.000000  0.000000      0.0  
2       0.001740  0.000000  0.000000  0.000000  0.000971  0.000000      0.0  
3       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000      0.0  
4       0.000012  0.000000  0.000000  0.000000  0.000000  0.000000      0.0  
...          ...       ...       ...       ...       ...       ...      ...  
305816  0.003045  0.000000  0.000000  0.000000  0.000728  0.000000      0.0  
305817  0.000870  0.000000  0.000000  0.000000  0.000000  0.000998      0.0  
305818  0.000000  0.000000  0.001406  0.000000  0.000000  0.000000      0.0  
305819  0.000000  0.001904  0.000000  0.000000  0.000000  0.000000      0.0  
305820  0.000000  0.001058  0.000000  0.000000  0.000000  0.000000      0.0  

[305821 rows x 13 columns]

In [79]:
# Save the current DataFrame
total_df.to_csv("phrase_frequency_no_df.csv")

In [109]:
# Calculate document frequency for every archetype
import math

total_df = pd.read_csv("phrase_frequency_no_df.csv", index_col=0)

for archetype in archetype_list:
    print(f"Archetype {archetype}:")
    
    twitter_subset = twitter_df.cleaned_text[twitter_df["archetype"] == archetype]
    
    # Select non-zero elements and calculate DF for every element
    for _, row in tqdm(total_df.iterrows()):
        if row[archetype] > 0.0:
            phrase = row[archetype]
            phrase_cnt = 0
            for line in twitter_subset:
                if phrase in line:
                    phrase_cnt += 1

            row[archetype] = row[archetype] * math.log(len(twitter_df) / (phrase_cnt + 1))      

1it [00:00,  6.74it/s]

Archetype artist:


305821it [01:11, 4270.99it/s]
1it [00:00,  7.50it/s]

Archetype caregiver:


305821it [02:06, 2426.88it/s]
1it [00:00,  7.31it/s]

...

305821it [00:57, 5354.71it/s] 
1it [00:00,  7.44it/s]

Archetype ruler:


305821it [01:14, 4095.34it/s]
1it [00:00,  7.12it/s]

Archetype seducer:


305821it [02:05, 2433.86it/s]


In [110]:
# Save the DataFrame with document frequency calculated
total_df.to_csv("phrase_frequency_with_df.csv")

In [111]:
# Print the new DataFrame
total_df

pattern    artist  caregiver  everyman  explorer      guru  \
0                ['aaron']  0.000000   0.000000       0.0  0.000786  0.000000   
1                ['abbie']  0.000000   0.000000       0.0  0.000000  0.000000   
2              ['ability']  0.000000   0.000000       0.0  0.001072  0.000793   
3         ['able', 'able']  0.000042   0.000000       0.0  0.000000  0.000000   
4       ['able', 'access']  0.000000   0.000000       0.0  0.000000  0.000000   
...                    ...       ...        ...       ...       ...       ...   
305816             ['zip']  0.000000   0.003239       0.0  0.000000  0.000000   
305817            ['zone']  0.001107   0.000000       0.0  0.000000  0.000000   
305818            ['zoom']  0.002029   0.000000       0.0  0.000000  0.001220   
305819              ['zu']  0.000000   0.000000       0.0  0.000000  0.000000   
305820             ['zum']  0.000000   0.000000       0.0  0.000000  0.000000   

            hero  innocent    jester  magician     rebel     ruler  seducer  
0       0.000653  0.000000  0.000000  0.000000  0.000000  0.000000      0.0  
1       0.000000  0.000000  0.000000  0.003622  0.000000  0.000000      0.0  
2       0.001740  0.000000  0.000000  0.000000  0.000971  0.000000      0.0  
3       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000      0.0  
4       0.000012  0.000000  0.000000  0.000000  0.000000  0.000000      0.0  
...          ...       ...       ...       ...       ...       ...      ...  
305816  0.003045  0.000000  0.000000  0.000000  0.000728  0.000000      0.0  
305817  0.000870  0.000000  0.000000  0.000000  0.000000  0.000998      0.0  
305818  0.000000  0.000000  0.001406  0.000000  0.000000  0.000000      0.0  
305819  0.000000  0.001904  0.000000  0.000000  0.000000  0.000000      0.0  
305820  0.000000  0.001058  0.000000  0.000000  0.000000  0.000000      0.0  

[305821 rows x 13 columns]